In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27 # figure size in inches

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 30) 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config Completer.use_jedi = False

import pingouin as pg

In [2]:
df_daybin_4 = pd.read_csv('CSV/df_daybin_4.csv')
df_daybin_4.head()

,id,days_bin,sbp,dbp,creatinine,glucose,ldl,hgb,atenolol,atorvastatin,bisoprolol,canagliflozin,carvedilol,dapagliflozin,irbesartan,...,metformin,metoprolol,nebivolol,olmesartan,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,race,gender,age,Stage_Progress
0,0,1,133.930,89.650,1.220,6.72,136.440000,13.45,0.0,910.0,0.0,0.0,0.0,0.0,0.0,...,91000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White,Male,70,1
1,0,2,125.080,65.970,1.440,7.01,100.521667,12.84,0.0,910.0,0.0,0.0,0.0,0.0,0.0,...,151000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White,Male,70,1
2,0,3,133.625,84.900,1.230,6.89,157.900000,13.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,242000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White,Male,70,1
3,0,4,154.280,84.290,1.155,5.70,72.116667,13.53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,182000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White,Male,70,1
4,1,1,155.525,77.435,1.785,9.96,89.570000,13.89,0.0,2440.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White,Female,71,0


In [3]:
# Based on value at days_bin = 4 (latest data)
df_daybin_4 = df_daybin_4[df_daybin_4['days_bin'] == 4]
df_daybin_4.shape

(292, 33)

# Compare between Stage_Progress = True / False

In [4]:
df_compare = df_daybin_4.groupby(['Stage_Progress']).agg(['mean', 'std']).round(1)
df_compare[['sbp', 'dbp', 'creatinine', 'glucose', 'ldl', 'hgb', 'age']]

sbp         dbp       creatinine      glucose        ldl  \
                 mean   std  mean   std       mean  std    mean  std  mean   
Stage_Progress                                                               
0               131.5  12.6  78.7   8.1        1.4  0.3     6.5  1.1  80.6   
1               136.1  15.2  80.7  10.3        1.3  0.4     7.1  1.9  89.8   

                       hgb        age       
                 std  mean  std  mean  std  
Stage_Progress                              
0               23.0  13.9  1.4  71.0  9.0  
1               27.7  13.4  1.5  69.2  9.8

In [5]:
def procttest(df):
    col_name = df.columns
    SID = df[col_name[0]] == 0 # Stage_Progress
    x = df.loc[SID, col_name[1]]
    y = df.loc[~SID, col_name[1]]
    result = pg.ttest(x, y)
    result = result.rename(index={'T-test' : col_name[1]})
    return result 
    
measure_list = ['sbp', 'dbp', 'creatinine', 'glucose', 'ldl', 'hgb', 'age']
for measure in measure_list:
    procttest(df_daybin_4[['Stage_Progress', measure]])


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
sbp,-2.583922,160.68222,two-sided,0.010658,"[-8.15, -1.09]",0.343238,3.165,0.784123


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
dbp,-1.667366,154.580308,two-sided,0.097466,"[-4.36, 0.37]",0.225152,0.509,0.437085


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
creatinine,0.670923,153.794271,two-sided,0.503276,"[-0.06, 0.12]",0.090797,0.169,0.112303


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
glucose,-2.956816,125.116256,two-sided,0.003717,"[-1.03, -0.2]",0.441476,8.284,0.942043


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
ldl,-2.800715,160.954654,two-sided,0.005724,"[-15.6, -2.7]",0.371772,5.454,0.844825


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
hgb,2.306903,175.434002,two-sided,0.022229,"[0.06, 0.78]",0.2957,1.68,0.657583


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
age,1.520263,174.972509,two-sided,0.13025,"[-0.54, 4.16]",0.195071,0.408,0.345072


In [6]:
df_pivot = df_daybin_4.pivot_table(index=['gender'], columns = 'Stage_Progress', aggfunc='size')
df_pivot = df_pivot.sort_values(0, ascending=False)
df_pivot
(df_pivot / df_pivot.sum() * 100).round(1)
expected, observed, stats = pg.chi2_independence(df_daybin_4, x='gender', y='Stage_Progress')
stats

Stage_Progress,0,1
gender,,
Female,129,42
Male,67,54


Stage_Progress,0,1
gender,,
Female,65.8,43.8
Male,34.2,56.2


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,12.036382,1.0,0.000522,0.203028,0.934400
1,cressie-read,0.666667,11.997438,1.0,0.000533,0.202700,0.933679
2,log-likelihood,0.000000,11.962911,1.0,0.000543,0.202408,0.933035
3,freeman-tukey,-0.500000,11.974388,1.0,0.000539,0.202505,0.933250
4,mod-log-likelihood,-1.000000,12.017633,1.0,0.000527,0.202870,0.934054
5,neyman,-2.000000,12.199759,1.0,0.000478,0.204402,0.937344


In [7]:
# After converting unknown to white
df_pivot =  df_daybin_4.pivot_table(index=['race'], columns = 'Stage_Progress', aggfunc='size')
df_pivot = df_pivot.sort_values(0, ascending=False)
df_pivot
(df_pivot / df_pivot.sum() * 100).round(1)
expected, observed, stats = pg.chi2_independence(df_daybin_4, x='race', y='Stage_Progress')
stats

Stage_Progress,0,1
race,,
White,163,82
Black,17,6
Asian,12,5
Hispanic,4,3


Stage_Progress,0,1
race,,
White,83.2,85.4
Black,8.7,6.2
Asian,6.1,5.2
Hispanic,2.0,3.1


C:\Users\SC-SG\AppData\Roaming\Python\Python38\site-packages\pingouin\contingency.py:151: UserWarning: Low count on observed frequencies.
  warnings.warn('Low count on {} frequencies.'.format(name))
C:\Users\SC-SG\AppData\Roaming\Python\Python38\site-packages\pingouin\contingency.py:151: UserWarning: Low count on expected frequencies.
  warnings.warn('Low count on {} frequencies.'.format(name))


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,0.927926,3.0,0.818684,0.056372,0.110497
1,cressie-read,0.666667,0.930006,3.0,0.818181,0.056435,0.110645
2,log-likelihood,0.000000,0.936225,3.0,0.816678,0.056624,0.111088
3,freeman-tukey,-0.500000,0.942696,3.0,0.815114,0.056819,0.111550
4,mod-log-likelihood,-1.000000,0.950729,3.0,0.813172,0.057061,0.112124
5,neyman,-2.000000,0.971582,3.0,0.808128,0.057683,0.113617
